# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [49]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import time
from types import SimpleNamespace

import no_trade as nt
import modelproject as mp
import numpy as np
import sympy as sm

from scipy import linalg
from scipy import optimize
from scipy.optimize import minimize

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
plt.rcParams.update({"axes.grid": True, "grid.color": "black",
                    "grid.alpha": "0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

The goods in this model are wine and cloth.

Find the optimal level of trade between England and Portugal, when Portugal has an absolut advantage in both goods, but England has a comparative one in one of the goods. 

The table below contains the number of hours each coutry need to put into make 1 of the two goods. 

| Country   | Cloth | Wine |      
|-----------|-------|------|      
| England   | 100   | 120  |           
| Portugal  | 90    | 80   |
     
| Country   | Cloth | Wine |
|-----------|-------|------|
| England   | 1     | 1.2  |
| Portugal  | 1     | 8/9  |

If no trade is happening, England requires 220 hours of work to produce one unit each good, and Portugal requires 170 hours of work to produce the same quantities.

If they do not trade and work 1000 hours, how much can they each consume? 


In [53]:
nt.portugal_production()
nt.england_production()

Portugal produces 6.25 units of wine and 5.56 units of cloth. And, the resulting utility level is 5.89
England produces 4.17 units of wine and 5.00 units of cloth. And, the resulting utility level is 4.56


Now, according to Ricardo if they were to trade with England they would be able to consume more of both output, given that both England and Portugal were to produce more of the good inwhich they had a comparative advantage. 

If they worked the same number of hours, and traded, taking advantage of their comparative advantages, what would the new comsumption bundels look like?

In [36]:
from scipy.optimize import minimize

# Define the objective function to maximize (i.e., the negative of the total production of both countries)


def objective(x):
    cloth_p = x[0]  # time spent by Portugal on cloth production
    cloth_e = x[1]  # time spent by England on cloth production

    # Calculate the production of wine and cloth for each country
    wine_p = (170 - cloth_p) * (8/9)  # production of wine by Portugal
    wine_e = (220 - cloth_e) * 1.2  # production of wine by England
    cloth_p = (170 - wine_p)  # production of cloth by Portugal
    cloth_e = (220 - wine_e)  # production of cloth by England

    # Return the negative of the total production as the objective to be minimized
    return -(wine_p + wine_e + cloth_p + cloth_e)

# Define the inequality constraints on x (the number of hours Portugal spends on cloth production)


def ineq_constraint1(x):
    return 170 - x[0]

# Define the inequality constraints on x (the number of hours England spends on cloth production)


def ineq_constraint2(x):
    return 220 - x[1]


# Define the bounds on x (the total number of hours Portugal spends on cloth production and England spends on cloth production)
bounds = [(0, 170), (0, 220), (0, 170), (0, 220)]

# Define the initial guess for x (the number of hours Portugal spends on cloth production and the number of hours England spends on cloth production)
x0 = [1, 2, 1, 2]

# Define the constraints dictionary
cons = [{'type': 'ineq', 'fun': ineq_constraint1},
        {'type': 'ineq', 'fun': ineq_constraint2}]

# Minimize the objective function subject to the constraints using the SLSQP algorithm
result = optimize.minimize(
    objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Extract the optimal values of x (the number of hours Portugal spends on cloth production and the number of hours England spends on cloth production)
x_opt = result.x[:2]

result
# # Extract the optimal values of wine and cloth production for both countries
# wine_p_opt = (170 - x_opt[0]) * (8/9)  # production of wine by Portugal
# cloth_p_opt = (170 - wine_p_opt)  # production of cloth by Portugal
# wine_e_opt = (220 - x_opt[1]) * 1.2  # production of wine by England
# cloth_e_opt = (220 - wine_e_opt)  # production of cloth by England

# # Print the results
# print("Portugal produces {:.2f} units of wine and {:.2f} units of cloth.".format(wine_p_opt, cloth_p_opt))
# print("England produces {:.2f} units of wine and {:.2f} units of cloth.".format(wine_e_opt, cloth_e_opt))


# # # Plot the contour plot of the objective function
# # x = np.linspace(0.8, 1.3, 100)
# y = np.linspace(0.8, 1.3, 100)
# X, Y = np.meshgrid(x, y)
# Z = objective(X)

# plt.figure()
# plt.contour(X, Y, Z, levels=30)
# plt.plot(result.x, objective(result.x), 'ro')
# plt.xlabel('Terms of trade (wine/cloth)')
# plt.ylabel('Terms of trade (cloth/wine)')
# plt.show()


     fun: -390.00000000000006
     jac: array([3.81469727e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([1., 2., 1., 2.])

In [5]:
import random

# Define the game structure and possible actions
actions = ["rock", "paper", "scissors"]
payoff_matrix = {
    ("rock", "rock"): (0, 0),
    ("rock", "paper"): (-1, 1),
    ("rock", "scissors"): (1, -1),
    ("paper", "rock"): (1, -1),
    ("paper", "paper"): (0, 0),
    ("paper", "scissors"): (-1, 1),
    ("scissors", "rock"): (-1, 1),
    ("scissors", "paper"): (1, -1),
    ("scissors", "scissors"): (0, 0)
}

# Initialize the game
state = ("unknown", "unknown")  # the initial state is unknown
player1_payoff = 0
player2_payoff = 0

while state[0] == "unknown" or state[1] == "unknown":
    # Agent 1 takes action
    player1_action = random.choice(actions)

    # Agent 2 takes action
    if state[1] == "unknown":
        player2_action = random.choice(actions)
    else:
        player2_action = max(
            actions, key=lambda x: payoff_matrix[(player1_action, x)][1])

    # Determine the outcome
    outcome = payoff_matrix[(player1_action, player2_action)]
    player1_payoff += outcome[0]
    player2_payoff += outcome[1]

    # Update the state
    if state[0] == "unknown":
        state = (player1_action, "unknown")
    else:
        state = (state[0], player2_action)

# Print the final payoffs and state
print("Final payoffs: Player 1 =", player1_payoff,
      ", Player 2 =", player2_payoff)
print("Final state:", state)


Final payoffs: Player 1 = 0 , Player 2 = 0
Final state: ('scissors', 'scissors')


## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 